# Librerías y Datos

Se instalan las librerías necesarias

In [ ]:
!pip install polyline
!pip install geopandas
!pip install ortools
!pip install pyautogui

In [ ]:
import pandas as pd
import numpy as np
import requests
import polyline
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from matplotlib import cm
import folium
from folium import Marker
import geopandas as gpd
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Se leen los datos de las 300 ubicaciones aleatorias con su correspondiente volumen

In [ ]:
df_ubicaciones=pd.read_csv("/content/drive/MyDrive/Universidad/Cuarto Semestre/Optimización Determinista/UbicacionesVolumen.csv")
df_ubicaciones

,Unnamed: 0,index,unidad,factura,foliocontrol,tienda,fechaventa,fechaenrutada,ruta,codigo,...,Longitud,Direccion,Codigo,DCF,CATEGORIAS,Articulo,Marca,Modelo,# Proveedor,Vol
0,0,23813,6359,4636,8082293,1448,26/01/2021,27/01/2021,84,361721,...,-99.179496,"MIGUEL HIDALGO, ESCANDO, calle AV PATRIOTISMO 88",361721,30104,RECAMARA,COLCHON,AMERICA,100-240 MODENA,60453,0.563538
1,1,23815,50423,7108,8082297,1056,26/01/2021,27/01/2021,86,352896,...,-99.245547,"NAUCALPAN, LOMA LINDA, calle BUGAMBILIA 45",352896,31012,RECAMARA,BASE,PUFFLAND,PALENCIA,65137,0.820800
2,2,23817,50423,33279,8082297,152,26/01/2021,27/01/2021,86,427979,...,-99.289361,"NAUCALPAN, LA OLIMPICA, calle RICARDO DELGADO 67",427979,40105,HOGAR,LOVE SEAT,MARBY,BARCELONA,61611,1.360000
3,3,23956,6071,957447,8082288,800,22/01/2021,27/01/2021,78,427979,...,-99.209470,"MIGUEL HIDALGO, ARGENTINA ANTIGUA, calle IGNAC...",427979,40105,HOGAR,LOVE SEAT,MARBY,BARCELONA,61611,1.360000
4,4,23821,51044,10158,8082280,834,26/01/2021,27/01/2021,74,204159,...,-99.266103,"ATIZAPAN DE ZARAGOZA, AMPL EMILIANO ZAPATA, ca...",204159,20408,TECNOLOGIA Y ENTRETENIMIENTO,AUDIO VERT,EVL,SE-12,87921,0.214957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,295,24753,54283,991518,8082265,800,26/01/2021,27/01/2021,4,407455,...,-99.116350,"GUSTAVO A MADERO, ESTRELLA, calle PROLONGACION...",407455,49201,RECAMARA,EDREDON,SUPREME,ELISA KS,92151,0.134097
296,296,24756,54283,992020,8082265,800,26/01/2021,27/01/2021,4,409081,...,-99.077394,"GUSTAVO A MADERO, OCEANO, calle APATZINGAN 25",409081,45902,HOGAR,TAPETE DE,STARHAUS,ARMONIA RED,76091,0.050449
297,297,24757,54324,15939,8082263,7696,19/01/2021,27/01/2021,2,151807,...,-99.268019,"NAUCALPAN, SAN MATEO NOPALA ZONA SUR, calle DE...",151807,18120,ENSERES DOMÉSTICOS,COMBOS,CONAIR,MS21COES,77038,0.006628
298,298,24762,54324,933815,8082263,800,19/01/2021,27/01/2021,2,164879,...,-99.258352,"NAUCALPAN, EMILIANO ZAPATA, calle CDA SANTA MA...",164879,14801,ENSERES DOMÉSTICOS,OLLAS DE P,T-FAL,92160PCN 6L,71218,0.037146


Se agrega la ubicación del CEDIS

Se cambia el volumen de m^3 a cm^3

In [ ]:
df_ubicaciones=df_ubicaciones.append({"index":00000,"Latitud":19.4907364,"Longitud":-99.1620038,"Direccion":"CEDIS Azcapotzcalco","Vol":0},ignore_index=True)
df_ubicaciones["Vol"]=round(df_ubicaciones["Vol"]*1000000)
df_ubicaciones["TiempoEntrega"]=0
df_ubicaciones.loc[df_ubicaciones['Vol']<=500000,"TiempoEntrega"]=5*60
df_ubicaciones.loc[(df_ubicaciones['Vol']>500000)&(df_ubicaciones['Vol']<=1000000),"TiempoEntrega"]=10*60
df_ubicaciones.loc[(df_ubicaciones['Vol']>1000000)&(df_ubicaciones['Vol']<=2000000),"TiempoEntrega"]=20*60
df_ubicaciones.loc[(df_ubicaciones['Vol']>2000000)&(df_ubicaciones['Vol']<=3000000),"TiempoEntrega"]=25*60
df_ubicaciones.loc[(df_ubicaciones['Vol']>3000000),"TiempoEntrega"]=30*60
df_ubicaciones

,Unnamed: 0,index,unidad,factura,foliocontrol,tienda,fechaventa,fechaenrutada,ruta,codigo,...,Direccion,Codigo,DCF,CATEGORIAS,Articulo,Marca,Modelo,# Proveedor,Vol,TiempoEntrega
0,0.0,23813,6359.0,4636.0,8082293.0,1448.0,26/01/2021,27/01/2021,84.0,361721.0,...,"MIGUEL HIDALGO, ESCANDO, calle AV PATRIOTISMO 88",361721.0,30104.0,RECAMARA,COLCHON,AMERICA,100-240 MODENA,60453.0,563538.0,600
1,1.0,23815,50423.0,7108.0,8082297.0,1056.0,26/01/2021,27/01/2021,86.0,352896.0,...,"NAUCALPAN, LOMA LINDA, calle BUGAMBILIA 45",352896.0,31012.0,RECAMARA,BASE,PUFFLAND,PALENCIA,65137.0,820800.0,600
2,2.0,23817,50423.0,33279.0,8082297.0,152.0,26/01/2021,27/01/2021,86.0,427979.0,...,"NAUCALPAN, LA OLIMPICA, calle RICARDO DELGADO 67",427979.0,40105.0,HOGAR,LOVE SEAT,MARBY,BARCELONA,61611.0,1360000.0,1200
3,3.0,23956,6071.0,957447.0,8082288.0,800.0,22/01/2021,27/01/2021,78.0,427979.0,...,"MIGUEL HIDALGO, ARGENTINA ANTIGUA, calle IGNAC...",427979.0,40105.0,HOGAR,LOVE SEAT,MARBY,BARCELONA,61611.0,1360000.0,1200
4,4.0,23821,51044.0,10158.0,8082280.0,834.0,26/01/2021,27/01/2021,74.0,204159.0,...,"ATIZAPAN DE ZARAGOZA, AMPL EMILIANO ZAPATA, ca...",204159.0,20408.0,TECNOLOGIA Y ENTRETENIMIENTO,AUDIO VERT,EVL,SE-12,87921.0,214957.0,300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,296.0,24756,54283.0,992020.0,8082265.0,800.0,26/01/2021,27/01/2021,4.0,409081.0,...,"GUSTAVO A MADERO, OCEANO, calle APATZINGAN 25",409081.0,45902.0,HOGAR,TAPETE DE,STARHAUS,ARMONIA RED,76091.0,50449.0,300
297,297.0,24757,54324.0,15939.0,8082263.0,7696.0,19/01/2021,27/01/2021,2.0,151807.0,...,"NAUCALPAN, SAN MATEO NOPALA ZONA SUR, calle DE...",151807.0,18120.0,ENSERES DOMÉSTICOS,COMBOS,CONAIR,MS21COES,77038.0,6628.0,300
298,298.0,24762,54324.0,933815.0,8082263.0,800.0,19/01/2021,27/01/2021,2.0,164879.0,...,"NAUCALPAN, EMILIANO ZAPATA, calle CDA SANTA MA...",164879.0,14801.0,ENSERES DOMÉSTICOS,OLLAS DE P,T-FAL,92160PCN 6L,71218.0,37146.0,300
299,299.0,24764,54324.0,934131.0,8082263.0,800.0,19/01/2021,27/01/2021,2.0,237903.0,...,"NAUCALPAN, U H UNIDAD Y PROGRESO, calle SAN MI...",237903.0,29502.0,CELULARES,PROTECTORE,BLOB,MOT ONE ZOOM DBL,77275.0,540.0,300


Se crea una matriz para los tiempos, otro para las distancias y otro para las rutas

# Cálculo de Tiempo entre cada par de puntos

Se unen todas las coordenadas de los datos en un solo String


In [ ]:
coordenadas=""
for i in range(len(df_ubicaciones)):
  fila=df_ubicaciones.iloc[i]
  long=fila["Longitud"]
  lat=fila["Latitud"]
  coordenadas=coordenadas+str(long)+","+str(lat)+";"
coordenadas=coordenadas[:-1]
coordenadas

'-99.1794964,19.4022809;-99.245547,19.4591888;-99.2893612,19.4432619;-99.2094698,19.4554826;-99.2661033,19.6012754;-99.2420559,19.4551017;-99.1758514,19.4769707;-99.1379909,19.4471603;-99.1563256,19.4269844;-99.2801071,19.4566583;-99.2664005,19.4550024;-99.2182511,19.5837955;-99.2517263,19.5668846;-99.1857698,19.4569065;-99.1872948,19.4036934;-99.2680195,19.5876287;-99.2496741,19.4395463;-99.2414616,19.4593702;-99.1787242,19.4796688;-99.1849803,19.481773;-99.0755706,19.493863;-99.1417065,19.4390061;-99.2007218,19.4752239;-99.1451175,19.4584726;-99.2998173,19.3568654;-99.0619259,19.4511712;-99.1118203,19.5028059;-99.2818438,19.4482962;-99.1384755,19.522558;-99.2335109,19.567857;-99.260528,19.4448435;-99.067733,19.4950679;-99.0731781,19.4495054;-99.1245349,19.4385273;-99.1256893,19.44234;-99.1398522,19.4184384;-99.1598279,19.5330315;-99.2623489,19.5900937;-99.3308954,19.4165628;-99.1296036,19.4239647;-99.1938371,19.5831351;-99.1067169,19.4888765;-99.235233,19.4542955;-99.1929203,19.58142

Se crea un Data Frame Vacío

In [ ]:
df_tiempos= pd.DataFrame()

Se calcula la distancia entre cada par de puntos. Se hace el request a la API de 10 en 10 filas para todas las columnas de la matriz

In [ ]:
for i in range(0,291,10):
  rangos= list(range(i,i+10))
  sources= ";".join(str(item) for item in rangos)
  url="http://router.project-osrm.org/table/v1/car/"+coordenadas+"?sources="+sources
  r = requests.get(url)
  res = r.json()
  df_tiempos=df_tiempos.append(res["durations"])
  print("Ubicaciones Completas:",i+10)
url="http://router.project-osrm.org/table/v1/car/"+coordenadas+"?sources=300"
r = requests.get(url)
res = r.json()
df_tiempos=df_tiempos.append(res["durations"])
df_tiempos

Ubicaciones Completas: 10
Ubicaciones Completas: 20
Ubicaciones Completas: 30
Ubicaciones Completas: 40
Ubicaciones Completas: 50
Ubicaciones Completas: 60
Ubicaciones Completas: 70
Ubicaciones Completas: 80
Ubicaciones Completas: 90
Ubicaciones Completas: 100
Ubicaciones Completas: 110
Ubicaciones Completas: 120
Ubicaciones Completas: 130
Ubicaciones Completas: 140
Ubicaciones Completas: 150
Ubicaciones Completas: 160
Ubicaciones Completas: 170
Ubicaciones Completas: 180
Ubicaciones Completas: 190
Ubicaciones Completas: 200
Ubicaciones Completas: 210
Ubicaciones Completas: 220
Ubicaciones Completas: 230
Ubicaciones Completas: 240
Ubicaciones Completas: 250
Ubicaciones Completas: 260
Ubicaciones Completas: 270
Ubicaciones Completas: 280
Ubicaciones Completas: 290
Ubicaciones Completas: 300


,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,300
0,0.0,740.2,1054.3,572.8,1608.7,747.3,617.2,563.2,353.9,1020.9,...,1210.3,814.4,1011.9,1221.3,857.8,1165.0,1028.6,810.3,907.9,870.6
1,649.9,0.0,468.1,298.9,1259.6,170.6,653.2,890.6,695.1,434.7,...,1470.8,1012.6,1240.4,1523.4,1033.7,1272.9,615.8,224.1,367.5,928.3
2,833.0,300.8,0.0,482.0,1316.2,425.4,836.3,1073.7,878.2,342.3,...,1653.9,1195.7,1423.5,1706.5,1216.8,1456.0,798.9,353.8,447.1,1111.4
3,610.2,395.5,709.6,0.0,1312.2,407.3,382.8,632.2,561.9,676.2,...,1200.4,742.2,970.0,1253.0,763.3,1123.1,724.4,465.6,567.9,678.6
4,1610.3,1297.0,1192.9,1271.0,0.0,1396.2,1339.1,1623.3,1593.7,1493.4,...,1961.3,1470.3,1736.4,1948.9,1513.7,1645.4,1154.2,1408.5,1568.2,1317.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,1035.1,1318.5,1632.6,1145.1,1701.0,1392.3,870.8,711.0,866.2,1599.2,...,401.1,489.1,248.4,494.6,463.8,0.0,1560.1,1388.6,1552.9,942.9
7,904.3,595.2,946.5,569.6,1270.4,694.4,857.9,1157.5,962.0,913.1,...,1696.1,1215.7,1471.2,1683.7,1259.1,1380.2,0.0,702.5,866.4,1092.0
8,786.1,224.4,444.0,435.1,1395.8,349.0,789.4,1026.8,831.3,410.6,...,1607.0,1148.8,1376.6,1659.6,1169.9,1409.1,752.0,0.0,459.4,1064.5
9,863.7,362.2,565.1,507.1,1538.1,265.0,861.4,1110.8,932.4,609.1,...,1679.0,1220.8,1448.6,1731.6,1241.9,1551.4,894.3,454.3,0.0,1135.4


Se cambia el index del dataframe para que coincida con los index de las ubicaciones

In [ ]:
df_tiempos.columns=df_ubicaciones["index"]
df_tiempos.index=df_ubicaciones["index"]
df_tiempos

index,23813,23815,23817,23956,23821,23822,23827,23830,23837,23838,...,24735,24740,24742,24750,24753,24756,24757,24762,24764,0
index,,,,,,,,,,,,,,,,,,,,,
23813,0.0,740.2,1054.3,572.8,1608.7,747.3,617.2,563.2,353.9,1020.9,...,1210.3,814.4,1011.9,1221.3,857.8,1165.0,1028.6,810.3,907.9,870.6
23815,649.9,0.0,468.1,298.9,1259.6,170.6,653.2,890.6,695.1,434.7,...,1470.8,1012.6,1240.4,1523.4,1033.7,1272.9,615.8,224.1,367.5,928.3
23817,833.0,300.8,0.0,482.0,1316.2,425.4,836.3,1073.7,878.2,342.3,...,1653.9,1195.7,1423.5,1706.5,1216.8,1456.0,798.9,353.8,447.1,1111.4
23956,610.2,395.5,709.6,0.0,1312.2,407.3,382.8,632.2,561.9,676.2,...,1200.4,742.2,970.0,1253.0,763.3,1123.1,724.4,465.6,567.9,678.6
23821,1610.3,1297.0,1192.9,1271.0,0.0,1396.2,1339.1,1623.3,1593.7,1493.4,...,1961.3,1470.3,1736.4,1948.9,1513.7,1645.4,1154.2,1408.5,1568.2,1317.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24756,1035.1,1318.5,1632.6,1145.1,1701.0,1392.3,870.8,711.0,866.2,1599.2,...,401.1,489.1,248.4,494.6,463.8,0.0,1560.1,1388.6,1552.9,942.9
24757,904.3,595.2,946.5,569.6,1270.4,694.4,857.9,1157.5,962.0,913.1,...,1696.1,1215.7,1471.2,1683.7,1259.1,1380.2,0.0,702.5,866.4,1092.0
24762,786.1,224.4,444.0,435.1,1395.8,349.0,789.4,1026.8,831.3,410.6,...,1607.0,1148.8,1376.6,1659.6,1169.9,1409.1,752.0,0.0,459.4,1064.5


Se calcula el tiempo de entrega del paquete en sí

In [ ]:
volumen5=df_ubicaciones[df_ubicaciones["Vol"]<=500000]["index"]
volumen10=df_ubicaciones[(df_ubicaciones["Vol"]>500000) &(df_ubicaciones["Vol"]<=1000000)]["index"]
volumen20=df_ubicaciones[(df_ubicaciones["Vol"]>1000000) &(df_ubicaciones["Vol"]<=2000000)]["index"]
volumen25=df_ubicaciones[(df_ubicaciones["Vol"]>2000000) &(df_ubicaciones["Vol"]<=3000000)]["index"]
volumen30=df_ubicaciones[df_ubicaciones["Vol"]>3000000]["index"]
volumen30

137    24156
138    24164
139    24439
250    24559
Name: index, dtype: int64

In [ ]:
df_tiempos[volumen5]=df_tiempos[volumen5]+(5*60)
df_tiempos[volumen10]=df_tiempos[volumen10]+(10*60)
df_tiempos[volumen20]=df_tiempos[volumen20]+(20*60)
df_tiempos[volumen25]=df_tiempos[volumen25]+(25*60)
df_tiempos[volumen30]=df_tiempos[volumen30]+(30*60)
df_tiempos

index,23813,23815,23817,23956,23821,23822,23827,23830,23837,23838,...,24735,24740,24742,24750,24753,24756,24757,24762,24764,0
index,,,,,,,,,,,,,,,,,,,,,
23813,600.0,1340.2,2254.3,1772.8,1908.7,1347.3,1817.2,1163.2,653.9,1620.9,...,1510.3,1114.4,1311.9,1521.3,1157.8,1465.0,1328.6,1110.3,1207.9,1170.6
23815,1249.9,600.0,1668.1,1498.9,1559.6,770.6,1853.2,1490.6,995.1,1034.7,...,1770.8,1312.6,1540.4,1823.4,1333.7,1572.9,915.8,524.1,667.5,1228.3
23817,1433.0,900.8,1200.0,1682.0,1616.2,1025.4,2036.3,1673.7,1178.2,942.3,...,1953.9,1495.7,1723.5,2006.5,1516.8,1756.0,1098.9,653.8,747.1,1411.4
23956,1210.2,995.5,1909.6,1200.0,1612.2,1007.3,1582.8,1232.2,861.9,1276.2,...,1500.4,1042.2,1270.0,1553.0,1063.3,1423.1,1024.4,765.6,867.9,978.6
23821,2210.3,1897.0,2392.9,2471.0,300.0,1996.2,2539.1,2223.3,1893.7,2093.4,...,2261.3,1770.3,2036.4,2248.9,1813.7,1945.4,1454.2,1708.5,1868.2,1617.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24756,1635.1,1918.5,2832.6,2345.1,2001.0,1992.3,2070.8,1311.0,1166.2,2199.2,...,701.1,789.1,548.4,794.6,763.8,300.0,1860.1,1688.6,1852.9,1242.9
24757,1504.3,1195.2,2146.5,1769.6,1570.4,1294.4,2057.9,1757.5,1262.0,1513.1,...,1996.1,1515.7,1771.2,1983.7,1559.1,1680.2,300.0,1002.5,1166.4,1392.0
24762,1386.1,824.4,1644.0,1635.1,1695.8,949.0,1989.4,1626.8,1131.3,1010.6,...,1907.0,1448.8,1676.6,1959.6,1469.9,1709.1,1052.0,300.0,759.4,1364.5


Se convierte la matriz de tiempos a una lista de listas

In [ ]:
tiemposMatrix= df_tiempos.values.tolist()

# Modelo de CRVP

Se establecen las capacidades de los vehículos

In [ ]:
capacidades=[20000000,20000000,20000000,20000000,20000000,20000000,20000000,20000000,20000000,20000000,20000000,20000000,20000000,20000000,20000000,20000000,20000000,20000000]

Las siguientes funciones corresponden al modelo CRVP que establece Google Developer en la siguiente página: https://developers.google.com/optimization/routing/cvrp 

In [ ]:
orden=[]
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = tiemposMatrix
    data['demands'] = df_ubicaciones['Vol']
    data['vehicle_capacities'] = capacidades
    data['num_vehicles'] = len(capacidades)
    data['depot'] = len(df_ubicaciones)-1
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        entregas=[]
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            entregas.append(node_index)
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Time of the route: {} seconds\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
        orden.append(entregas)
    print('Total time of all routes: {} seconds'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))


def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)
    orden=[]


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.FromSeconds(30)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)
    
    return(routing)


routing= main()

Objective: 228552
Route for vehicle 0:
 300 Load(0.0) ->  228 Load(677740.0) ->  233 Load(805440.0) ->  181 Load(821740.0) ->  127 Load(1501473.0) ->  175 Load(1771473.0) ->  173 Load(1818415.0) ->  119 Load(1960089.0) ->  92 Load(2036251.0) ->  255 Load(2595316.0) ->  250 Load(5845316.0) ->  253 Load(6402315.0) ->  193 Load(7480435.0) ->  77 Load(7750435.0) ->  180 Load(7973560.0) ->  190 Load(7975719.0) ->  200 Load(8052519.0) ->  201 Load(8269710.0) ->  275 Load(8877326.0) ->  185 Load(9209826.0) ->  191 Load(9289246.0) ->  276 Load(9293551.0) ->  76 Load(9563551.0) ->  184 Load(9567419.0) ->  186 Load(9622664.0) ->  189 Load(9622677.0) ->  29 Load(11236989.0) ->  146 Load(13046989.0) ->  285 Load(13841289.0) ->  283 Load(15091289.0) ->  286 Load(15459429.0) ->  164 Load(16268364.0) ->  11 Load(17205210.0) ->  40 Load(17968910.0) ->  226 Load(18251468.0) ->  43 Load(19541468.0) ->  278 Load(19544920.0) ->  277 Load(19825440.0) ->  188 Load(19884800.0) ->  86 Load(19884800.0) ->  300

Se obtienen las rutas sólo de los camiones usados, y se agrega el punto final al que llegan

In [ ]:
ordenp=[]
capacidadF=[]
for i in range(len(orden)):
  temp1=orden[i]
  if(len(temp1)!=1):
    temp= temp1
    temp.append(temp[0])
    ordenp.append(temp)
    capacidadF.append(capacidades[i])
for j in range(len(ordenp)):
  i=ordenp[j]
  print("Camión",j,"hace",len(i),"paradas")

Camión 0 hace 41 paradas
Camión 1 hace 35 paradas
Camión 2 hace 47 paradas
Camión 3 hace 43 paradas
Camión 4 hace 29 paradas
Camión 5 hace 44 paradas
Camión 6 hace 37 paradas
Camión 7 hace 40 paradas


Se crea una lista de dataframes correspondiente a las ubicaciones que sigue cada camión. Cada dataframe corresponde a un camión

In [ ]:
listdf_enrutado=[]
for i in ordenp:
  df_enrutado=pd.DataFrame(columns=df_ubicaciones.columns)
  df_enrutado=df_enrutado.append(df_ubicaciones.iloc[i])
  df_enrutado.reset_index(inplace=True)
  listdf_enrutado.append(df_enrutado)
print("Cantidad de dataframes:",len(listdf_enrutado))

Cantidad de dataframes: 8


Se imprime el volumen usado y total de cada uno de los camiones usados

In [ ]:
for i in range(len(listdf_enrutado)):
  df_temp=listdf_enrutado[i]
  sum_volumen=df_temp["Vol"].sum()
  print("Camión ",i,"volumen=",sum_volumen,"cm^3 de",capacidadF[i],"cm^3")

Camión  0 volumen= 19884800.0 cm^3 de 20000000 cm^3
Camión  1 volumen= 19751088.0 cm^3 de 20000000 cm^3
Camión  2 volumen= 19718570.0 cm^3 de 20000000 cm^3
Camión  3 volumen= 19795912.0 cm^3 de 20000000 cm^3
Camión  4 volumen= 19658456.0 cm^3 de 20000000 cm^3
Camión  5 volumen= 19710903.0 cm^3 de 20000000 cm^3
Camión  6 volumen= 18778238.0 cm^3 de 20000000 cm^3
Camión  7 volumen= 19747656.0 cm^3 de 20000000 cm^3


Se crea una lista de dataframes correspondiente a las rutas que sigue cada camión. Cada dataframe corresponde a un camión

In [ ]:
listdf_rutasFinales=[]
indexes=df_ubicaciones["index"]
for j in ordenp:
  df_rutasFinales=pd.DataFrame(columns=["Inicio","Final"])
  for i in range(len(j)-1):
    inicio=i
    indexInicio=indexes[j[inicio]]
    final=i+1
    indexFinal=indexes[j[final]]
    fila={"Inicio":indexInicio,"Final":indexFinal}
    df_rutasFinales=df_rutasFinales.append(fila,ignore_index=True)
  listdf_rutasFinales.append(df_rutasFinales)
print("Cantidad de Dataframes:",len(listdf_rutasFinales))

Cantidad de Dataframes: 8


Se calcula la ruta que sigue cada camión usando un request a la API para cada par de ubicaciones

In [ ]:
list_columnaRutas=[]
list_columnaTiempos=[]
contador=0
for i in ordenp:
  columnaRutas=[]
  columnaTiempos=[]
  for j in range(len(i)-1):
    ubicacion1=df_ubicaciones.iloc[i[j]]
    ubicacion2=df_ubicaciones.iloc[i[j+1]]
    coordenadas=str(ubicacion1["Longitud"])+","+str(ubicacion1["Latitud"])+";"+str(ubicacion2["Longitud"])+","+str(ubicacion2["Latitud"])
    url="http://router.project-osrm.org/route/v1/foot/"+coordenadas
    r = requests.get(url)
    res = r.json()
    columnaRutas.append(polyline.decode(res["routes"][0]["geometry"]))
    columnaTiempos.append(res["routes"][0]["duration"])
  list_columnaRutas.append(columnaRutas)
  list_columnaTiempos.append(columnaTiempos)
  contador+=1
  print("Camión",contador,"Terminado")

Camión 1 Terminado
Camión 2 Terminado
Camión 3 Terminado
Camión 4 Terminado
Camión 5 Terminado
Camión 6 Terminado
Camión 7 Terminado
Camión 8 Terminado


Se agrega la columna de tiempo y de ruta a cada uno de los dataframes

In [ ]:
for i in range(len(listdf_rutasFinales)):
  df_temp= listdf_rutasFinales[i]
  df_temp["Ruta"]=list_columnaRutas[i]
  df_temp["Tiempo"]=list_columnaTiempos[i]
  listdf_rutasFinales[i]=df_temp
print("Cantidad de Dataframes:",len(listdf_rutasFinales))

Cantidad de Dataframes: 8


Se muestra el tiempo que hace cada ruta

In [ ]:
for i in range(len(listdf_rutasFinales)):
  df_temp=listdf_rutasFinales[i]
  sum_tiempo=df_temp["Tiempo"].sum()
  print("Camión ",i,"tiempo=",round(sum_tiempo),"segundos |", round(sum_tiempo/60),"minutos")

Camión  0 tiempo= 7362 segundos | 123 minutos
Camión  1 tiempo= 6638 segundos | 111 minutos
Camión  2 tiempo= 10934 segundos | 182 minutos
Camión  3 tiempo= 8686 segundos | 145 minutos
Camión  4 tiempo= 5205 segundos | 87 minutos
Camión  5 tiempo= 11147 segundos | 186 minutos
Camión  6 tiempo= 5295 segundos | 88 minutos
Camión  7 tiempo= 8129 segundos | 135 minutos


Se añade una función que obtiene una lista de colores basados en un gradiente

In [ ]:
def gradientColors(nColors,gradientN):
  gradiente=cm.get_cmap(gradientN,256)
  listaColores=[]
  for i in np.linspace(0,1,nColors):
    listaColores.append(gradiente(i))
  return(listaColores)

Se obtiene una lista de colores, un color para cada dataframe, tanto el de ubicaciones como el de rutas

In [ ]:
listaColores= gradientColors(len(listdf_enrutado),"gist_rainbow")
listaColoresHex=[]
for i in range(len(listaColores)):
  colorTuple= listaColores[i]
  colorHex= '#%02x%02x%02x' % tuple((np.array(colorTuple)*255).astype(int)[:-1])
  listaColoresHex.append(colorHex)
listaColoresHex

['#ff0028',
 '#ff9900',
 '#9cff00',
 '#00ff25',
 '#00ffec',
 '#004dff',
 '#7b00ff',
 '#ff00bf']

A cada dataframe se le pone su color correspondiente

In [ ]:
for i in range(len(listdf_enrutado)):
  df_tempEnrutado=listdf_enrutado[i]
  df_tempEnrutado["Color"]=listaColoresHex[i]
  listdf_enrutado[i]=df_tempEnrutado
  df_tempRutas=listdf_rutasFinales[i]
  df_tempRutas["Color"]=listaColoresHex[i]
  listdf_rutasFinales[i]=df_tempRutas
  print("Camion",i,"es de color",listaColoresHex[i])

Camion 0 es de color #ff0028
Camion 1 es de color #ff9900
Camion 2 es de color #9cff00
Camion 3 es de color #00ff25
Camion 4 es de color #00ffec
Camion 5 es de color #004dff
Camion 6 es de color #7b00ff
Camion 7 es de color #ff00bf


Se crea un mapa con cada una de las ubicaciones y el CEDIS

In [ ]:

map= folium.Map(location=[19.4907364,-99.1620038],zoom_start=12, tiles="cartodbpositron")
for j in listdf_enrutado:
  for index, row in j.iloc[:-1].iterrows():
    folium.CircleMarker(location=[row["Latitud"],row["Longitud"]],popup=str(index)+": "+row["Direccion"],
                        fill=True,fill_color=row["Color"],color = row["Color"],fill_opacity=0.7, 
                        tooltip=("Tiempo Entrega: "+str(row["TiempoEntrega"]/60)+" minutos")).add_to(map)


Marker([19.4907364,-99.1620038],icon=folium.Icon(color="darkblue",icon_color="white",icon="glyphicon-home"),tooltip="CEDIS Azcapotzalco").add_to(map)
map

Se crea un mapa con las rutas que sigue cada camión. Cada ruta empieza en rojo y acaba en morado

In [ ]:
for i in listdf_rutasFinales:
  for index, row in i.iterrows():
    folium.PolyLine(row["Ruta"],weight=8,color=row["Color"],opacity=0.6,tooltip=str(index)+": Tiempo enrutamiento "+str(round(row["Tiempo"]/60))+" minutos").add_to(map)
map

In [ ]:
map.save("CRVP_Final_TiemposEntrega.html")